# Accessing *NHLBI BioData Catalyst® (BDC)* Harmonized variables using R PIC-SURE API

This tutorial notebook will demonstrate how to query and work with the *BDC* cross-studies harmonized variables using R PIC-SURE API. For a more step-by-step introduction to the R PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to review the \"Get your security token\" documentation, which exists in the [`README.md` file](../README.md). It explains about how to get a security token, which is mandatory to access the databases.**

 -------

# Environment set-up

## System requirements
- R 3.4 or later

## Install Packages

**Note that if you are using the dedicated PIC-SURE environment within the *BDC Powered by Seven Bridges (BDC-Seven Bridges)* platform, the necessary packages have already been installed.**

*Troubleshooting: if you encounter an error while installing packages or connecting to a PIC-SURE resource, try restarting your kernel and trying again.*

In [ ]:
### Uncomment this code if you are not using the PIC-SURE environment in *BDC-Seven Bridges*), or if you do not have all the necessary dependencies installed.
#install.packages(c("devtools", "dplyr", "ggplot2"))

In [ ]:
devtools::install_github("hms-dbmi/pic-sure-r-client", ref="master", force=T, quiet=TRUE)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", ref="master", force=T, quiet=TRUE)
devtools::install_github("hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds", ref="master", force=T)
library(hpds)
library(dplyr)
library(ggplot2)


## Connecting to a PIC-SURE resource

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
connection <- picsure::connect(PICSURE_network_URL, token)
authPicSure = bdc::use.authPicSure(connection)

## About Harmonized Variables

The data harmonization effort aims to produce "a high quality, lasting resource of publicly available and thoroughly documented harmonized phenotype variables". The TOPMed Data Coordinating Center collaborates with Working Group members and phenotype experts on this endeavor. So far, 44 harmonized variables are accessible through PIC-SURE API (in addition to the age at which each variable value as been collected for a given subject).

The following phenotypes are included as harmonized variables:

- Key NHLBI phenotypes    
    - Blood cell counts
    - VTE
    - Atherosclerosis-related phenotypes
    - Lipids
    - Blood pressure


- Common covariates
    - Height
    - Weight
    - BMI
    - Smoking status
    - Race/ethnicity

More information about the variable harmonization process is available at https://www.nhlbiwgs.org/sites/default/files/pheno_harmonization_guidelines.pdf

# Working with the TOPMed DCC Harmonized Variables in PIC-SURE
## 1. Identifying harmonized variables of interest
First, let's explore what harmonized variables are available in PIC-SURE by searching for the keyword `harmonized`.

In [ ]:
dictionary <- bdc::use.dictionary(connection) # set up the variable dictionary
harmonized_dic <- bdc::find.in.dictionary(dictionary, "harmonized")
harmonized_df <- bdc::extract.dataframe(harmonized_dic)
head(harmonized_df)


We can see that these variables are all part of the unique study ID pertaining to the DCC Harmonized dataset.

In [ ]:
unique(harmonized_df$study_id)

We can also see that although there are only 44 DCC Harmonized variables, we have found 125 'harmonized' variables in PIC-SURE. This is because our variable results include subject IDs and 'metadata variables' which contain information about the age of the subject when a certain measure was taken, or the units of a variables. Let's exclude these.

In [ ]:
dim(harmonized_df)

In [ ]:
# Discarding "subject ID",
# the variables which only indicate age of the subject at which a given harmonized variable was been measured,
# and variables which indicate the units of a given harmonized variable
harmonized_df <- harmonized_df[!grepl(pattern = "age_at|SUBJECT_ID|unit_", harmonized_df$var_name),]
dim(harmonized_df)
head(harmonized_df[,c('var_name', 'var_description', 'HPDS_PATH', 'study_id')])

We can now see our 44 harmonized variables. This is in line with the [DCC Harmonized Variables documentation](https://github.com/UW-GAC/topmed-dcc-harmonized-phenotypes). 

## 2. Selecting variables and retrieving data from the database

Let's say we are interested in the subset of Harmonized Variables pertaining to patient demographics. 

We might do this by selecting variables based on what **datatable** or **variable group** the variables belong to. 

We will do this by filtering on the `group_id` column. We can see the values of this column and how many variables are in each group:

In [ ]:
table(harmonized_df$group_id)

Since we are interested in patient demographics, we filter our dataframe to include all harmonized variables which are part of the `demographic` variable group or data table. 

In [ ]:
demographic_df <- harmonized_df[grepl('demographic', harmonized_df$group_id),]
demographic_df[,c('var_name', 'var_description', 'HPDS_PATH', 'study_id')]
vars_of_interest <- demographic_df$HPDS_PATH

## Query PIC-SURE for participant-level data for harmonized variables of interest

In [ ]:
demographic_query <- bdc::new.query(authPicSure) # Start a new query
vars_of_interest <- c(vars_of_interest, '\\DCC Harmonized data set\\demographic\\race_1\\')
invisible(lapply(vars_of_interest, bdc::query.anyof.add, query = demographic_query))
demographic_results <- bdc::query.run(demographic_query)
head(demographic_results)

## Visualizing our sex and race harmonized variables across study cohorts
### Male:Female ratio across study cohorts

Below, we wrangle the data to calculate the male/female sex ratio per study cohort and prepare our data for plotting.


In [ ]:
mf_df <- demographic_results %>%
    select(grep('sex|subcohort', colnames(demographic_results)))
colnames(mf_df) <- c('sex', 'subcohort')
mf_df <- mf_df %>%
    group_by(subcohort) %>%
    filter(subcohort != '') %>% #filter out empty subcohorts
    summarise(female = sum(sex == 'Female'),
              male = sum(sex == 'Male')) %>%
    mutate(mf_ratio = male / female)

head(mf_df)

In [ ]:
options(repr.plot.width=16, repr.plot.height=12)
mf_df %>%
ggplot(aes(x = subcohort, y = mf_ratio)) + 
    geom_col() + 
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 12)) +
    labs(x = 'Study Cohort', y = 'Male/Female ratio', title = 'Male/Female ratio across study cohorts')
    

### Participant race percentages across study cohorts
Below, we wrangle the data to calculate the percentage of participants in given racial categories per study cohort and prepare out data for plotting.

In [ ]:
race_df <- demographic_results %>%
    select(grep('race|subcohort', colnames(demographic_results)))
colnames(race_df) <- c('race', 'subcohort')
race_df <- race_df %>%
    filter(subcohort != '', race != '') %>% # filter out blanks
    group_by(subcohort, race) %>%
    summarise(n = n())
head(race_df)

In [ ]:
ggplot(race_df, aes(fill=race, y=n, x=subcohort)) + 
    geom_bar(position="fill", stat="identity") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 12)) +
    labs(x = 'Study Cohort', y = 'Race percentage', title = 'Race percentage distribution across study cohorts')
    